### Importação Bibliotecas

---

In [56]:
import mlflow
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time


### Constantes

---

In [57]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "default.payment.next.month"


Definir a diretoria onde as experiências são guardadas

In [58]:
from pathlib import Path

uri = "../../mlruns"

Path(uri).mkdir(parents=True, exist_ok=True)

mlflow.set_tracking_uri(uri)

In [59]:
mlflow.set_experiment("Rumos Bank Lending")

<Experiment: artifact_location=('file:c:/Users/PereiraRodrigues/Documents/Data Science Pos Graduate/Oper '
 'ML/OML-trabalho/rumos_bank/notebooks/mlflow/../../mlruns/885472256422112193'), creation_time=1743551416007, experiment_id='885472256422112193', last_update_time=1743551416007, lifecycle_stage='active', name='Rumos Bank Lending', tags={}>

Dataset

In [60]:
df = pd.read_csv(ROOT_PATH + 'lending_data.csv')
df = df.sample(frac=0.01, random_state=SEED)

In [61]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = SEED)

In [62]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

### Criar run mlflow

In [63]:
run = mlflow.start_run(run_name="Ml Model Run - 1 - pipeline")
RUN_ID = run.info.run_uuid
RUN_ID

'1f28a7ede98f4dbb9a5305a85b9c5a74'

### Guardar datasets, modelos, artefactos, métricas e parametros da run

In [64]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

# Guardamos a seed utilizado como parametro
mlflow.log_param("seed", SEED)

c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigue

42

In [65]:
models_and_params = [

    ("logistic_regression",
     Pipeline([
         ("Classifier", LogisticRegression(max_iter=500, solver='lbfgs', random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__C": [0.001, 0.01, 0.1, 1, 10, 100]
     }),

    ("knn",
     Pipeline([
         ("Classifier", KNeighborsClassifier())
     ]),
     {
         "Classifier__n_neighbors": list(range(1, 10))
     }),


    ("decision_tree",
     Pipeline([
         ("Classifier", tree.DecisionTreeClassifier(random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__max_depth": [3, 6],
         "Classifier__min_samples_split": [2, 4, 10]
     })
]

In [66]:
for model_name, pipeline, param_grid in models_and_params:
    print(f"Treinando modelo e guardando todos os logs em MlFlow: {model_name}")
    #with mlflow.start_run(run_name=f"{model_name}_run"):
    start_time = time.time()
        # GridSearchCV para encontrar os melhores parâmetros
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    end_time = time.time()
    elapsed_time = end_time - start_time  # tempo em segundos

    best_model = grid_search.best_estimator_

    # Log do modelo no MLflow
    mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path=model_name,
            registered_model_name=model_name
    )

        # Log dos parâmetros e métrica principal
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("best_cv_score", grid_search.best_score_)
    mlflow.log_metric("training_time_sec", elapsed_time)  # tempo registado

    print(f"Modelo '{model_name}' guardado  MLflow.")

Treinando modelo e guardando todos os logs em MlFlow: logistic_regression


c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/06 22:30:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'logistic_regression' already exists. Creating a new version of this model...
Created version '4' of model 'logistic_regression'.


Modelo 'logistic_regression' guardado  MLflow.
Treinando modelo e guardando todos os logs em MlFlow: knn


2025/04/06 22:30:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'knn' already exists. Creating a new version of this model...
Created version '4' of model 'knn'.


Modelo 'knn' guardado  MLflow.
Treinando modelo e guardando todos os logs em MlFlow: decision_tree


2025/04/06 22:30:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'decision_tree' already exists. Creating a new version of this model...
Created version '3' of model 'decision_tree'.


Modelo 'decision_tree' guardado  MLflow.


In [67]:
# for model_name, pipeline, param_grid in models_and_params:
#     print(f"Treinando modelo e guardando todos os logs em MlFlow: {model_name}")
#     with mlflow.start_run(run_name=f"{model_name}_run"):
#         start_time = time.time()
#         # GridSearchCV para encontrar os melhores parâmetros
#         grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
#         grid_search.fit(X_train, y_train)

#         end_time = time.time()
#         elapsed_time = end_time - start_time  # tempo em segundos

#         best_model = grid_search.best_estimator_

#         # Log do modelo no MLflow
#         mlflow.sklearn.log_model(
#             sk_model=best_model,
#             artifact_path=model_name,
#             registered_model_name=model_name
#         )

#         # Log dos parâmetros e métrica principal
#         mlflow.log_params(grid_search.best_params_)
#         mlflow.log_metric("best_cv_score", grid_search.best_score_)
#         mlflow.log_metric("training_time_sec", elapsed_time)  # tempo registado

#         print(f"Modelo '{model_name}' guardado  MLflow.")

In [68]:
mlflow.end_run()

In [69]:
run = mlflow.get_run(RUN_ID)

In [70]:
run.data

<RunData: metrics={'best_cv_score': 0.675, 'training_time_sec': 0.14899420738220215}, params={'Classifier__C': '0.001',
 'Classifier__max_depth': '6',
 'Classifier__min_samples_split': '2',
 'Classifier__n_neighbors': '8',
 'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "1f28a7ede98f4dbb9a5305a85b9c5a74", '
                             '"artifact_path": "logistic_regression", '
                             '"utc_time_created": "2025-04-06 '
                             '22:30:31.146932", "model_uuid": '
                             '"15381c8ae7504b63b065da36adf1102a", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.9", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                        